# Build commit table

The overall goal is to pull commit information about a single project from github, then format that info as a table where each row is a separate commit. Here is the target, where each row represents a single commit:

<img src="https://www.dropbox.com/s/om587xsel6duayq/Screenshot%202019-06-25%2012.20.00.png?raw=1">

I first tried to run this from colab. I was able to download the hpcl folder to my desktop from here: https://github.com/HPCL/ideas-uo/tree/master/code/hpcl. Then upload it with code below. I first zipped it on the desktop so you can see unzipping below.

However, I failed to bring in mysql.connector from colab so switched to jupyter server on my desktop

## Make sure up to date



In [1]:
#~/ideas-uo/machine_learning/predicting-daily-commits
%pwd

u'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/machine_learning/predicting-daily-commits'

In [2]:
#move up to uo-ideas folder - must be better way
%cd ../..

/Users/fickas/Dropbox/boyana/my_work/ideas-uo


In [3]:
#Refresh local repository
!git pull

Already up-to-date.


In [4]:
import os
repository_dir = os.getcwd() #~/ideas-uo/
library_dir = repository_dir + '/code/hpcl'  #I am unable to import hpcl folder and don't know why
library_dir

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code/hpcl'

In [5]:

import sys
sys.path.append(library_dir)


In [6]:
print( '\n'.join(sys.path))


/Users/fickas/anaconda2/lib/python27.zip
/Users/fickas/anaconda2/lib/python2.7
/Users/fickas/anaconda2/lib/python2.7/plat-darwin
/Users/fickas/anaconda2/lib/python2.7/plat-mac
/Users/fickas/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages
/Users/fickas/anaconda2/lib/python2.7/lib-tk
/Users/fickas/anaconda2/lib/python2.7/lib-old
/Users/fickas/anaconda2/lib/python2.7/lib-dynload
/Users/fickas/.local/lib/python2.7/site-packages
/Users/fickas/anaconda2/lib/python2.7/site-packages
/Users/fickas/anaconda2/lib/python2.7/site-packages/Mako-1.0.7-py2.7.egg
/Users/fickas/anaconda2/lib/python2.7/site-packages/aeosa
/Users/fickas/anaconda2/lib/python2.7/site-packages/sputnik-0.9.3-py2.7.egg
/Users/fickas/anaconda2/lib/python2.7/site-packages/IPython/extensions
/Users/fickas/.ipython
/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code/hpcl


## Not happy

I am having to import each of the files separately because could not import hpcl folder - yuck.

However, I do note that the imports within these files seem to work, e.g., import hpcl.Command as Command. But can't get it to work from notebook.


In [7]:
import os, sys, subprocess
import Command
import GitCommand
#import mysql.connector    #works from command line but not here
from analysis import *

## Biggish issue

Jason's code will not run in 3.6 so I switched kernels to 2.7. Reluctantly! Like to get everyhing in 3.6 soon.

Note I did have to create a tmp dir prior to running this.

In [14]:
#move back to code dir

os.chdir(repository_dir + '/code')
os.getcwd()

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code'

In [15]:
#Thanks to Jason for this code

def checkoutSubrepos(repos,tdir):
    #currdir = os.getcwd()
    for repopath in repos.keys():
        if repopath == tdir: continue
        retcode, out, err = Command.Command('git checkout %s%s' % repos[repopath]).run(dryrun=dry_run)
    return


if __name__ == "__main__":
    
    print('Going to load repos...')

    #Load the list of repos to clone from the following file
    urls = open(os.path.join('.','URL.txt'),'r').readlines()

    #Move to the tmp directory that will hold the repos
    currdir = os.getcwd()
    tmpdir = os.path.join(currdir,'tmp')  #sff: fails if does not already exist
    #os.chdir(tmpdir)

    commander = GitCommand.GitCommand(tmpdir)

    #Download each repo
    for url in urls:
        print('Cloning: ' + url)
        commander.cloneRepo(url)

    print('Finished cloning repos.')

    outfile = open('../stats.csv','w')
    outfile.write('Date' + ',' + ','.join(categories.keys()) + ',' + 'Other' + '\n')

    #Setup DB connection, this will probably change soon.
    #mydb = mysql.connector.connect(host="localhost", port="3307", user="pythondb", passwd="********", database="gitstats")
    #print(mydb)
    #mycursor = mydb.cursor()
    #sql = "INSERT INTO stats (reponame, stats) VALUES(%s, %s)"

    #Now run commands on the repos
    for repo in os.listdir('.'):

        print('Checking: ' + repo)
        prefix,versions = commander.getRepoVersions(repo)
        print(prefix)
        print(versions)

        #Gets all the commit data in a dictionary format {authorname:{'total_commits':0, 'commits':{'date':'', 'message':'', 'diffs':{'filename':'', 'diff':''}}}}
        commits = commander.getRepoCommitData(repo)

        print(len(commits))
        for key,values in commits.items():
            print(key)
            #print(values)

        """ stats = getStats('.',repo)
        ts = repo + ', ' + out.strip().split('-')[0] 
        buf = ts
        for i in range(0,len(category_names)): 
            buf += ', %d' % stats[category_names[i]]
            linecounts[i].append(stats[category_names[i]]*0.001)
        outfile.write(buf+'\n')
        #years.append(int(ts))
        print(stats)
        
        val = (repo, buf)"""
        #mycursor.execute(sql,val)
        #mydb.commit()

    outfile.close()




Going to load repos...
Cloning: https://github.com/spack/spack.git

Cloning: https://github.com/lanl/LATTE.git

Finished cloning repos.
Checking: LATTE

['cf192f6fd4209d7dc0759321451c30ccf5d51b02', '79d950d1f714c1358a018210f8f2603edef84520', 'b191c3c3fbfb699bded53435611d24f27fffcd2f', '8eb5f8188764116ec3a1690233cee51a5d636583']
git checkout cf192f6fd4209d7dc0759321451c30ccf5d51b02

git checkout 79d950d1f714c1358a018210f8f2603edef84520

git checkout b191c3c3fbfb699bded53435611d24f27fffcd2f

git checkout 8eb5f8188764116ec3a1690233cee51a5d636583

15
Christoph Junghans <junghans@votca.org>
Christian Negre <cnegre@lanl.gov>
Marc Cawkwell <cawkwell@lanl.gov>
cawkwell <cawkwell@gransasso.lanl.gov>
Christoph Junghans <junghans@lanl.gov>
rperriot <rperriot@lanl.gov>
Nestor F. Aguirre <nfaguirrec@gmail.com>
Christoph Junghans <christoph.junghans@gmail.com>
Axel Kohlmeyer <akohlmey@gmail.com>
a1k12 <a1k2112@gmail.com>
Chang Liu <chliu1990@users.noreply.github.com>
Christian F. A. Negre <christian

## Let's see what we have

In [16]:
len(commits)  #50 separate people doing commits on the project

465

In [18]:
type(commits)

dict

In [19]:
tups = commits.items()  #get it in form easier to see

In [20]:
tups[0]

('Ian Lee <IanLee1521@gmail.com>',
 {'commits': [{'date': 'Tue Sep 6 14:52:44 2016 -0700',
    'diffs': [{'diff': ' ',
      'filename': ',7 +20,7 @@ written in pure Python, and specs allow package authors to write a'}],
    'message': '    Fixed broken link in README (#1733)\n'},
   {'date': 'Tue Sep 6 14:52:44 2016 -0700',
    'diffs': [{'diff': ' ',
      'filename': ',7 +20,7 @@ written in pure Python, and specs allow package authors to write a'}],
    'message': '    Fixed broken link in README (#1733)\n'},
   {'date': 'Thu Jun 28 13:32:43 2018 -0700',
    'diffs': [{'diff': ' ',
      'filename': ',7 +31,8 @@ class Nmap(AutotoolsPackage):'}],
    'message': '    Added nmap version 7.70 (#8606)\n'},
   {'date': 'Tue Sep 6 14:52:44 2016 -0700',
    'diffs': [{'diff': ' ',
      'filename': ',7 +20,7 @@ written in pure Python, and specs allow package authors to write a'}],
    'message': '    Fixed broken link in README (#1733)\n'}],
  'total_commits': 4})

In [ ]:
x = tups[1]
x

In [ ]:
x[1]['commits']

I am going to reformat the data. Currently the key is user and the value is a nested structure that includes all the commits for that person. I want to invert this. I'd like a list of commits, where each commit has info including person.

Note Python is a mess when it comes to time zones. What is supposed to work, %z, does not. See discussion here: https://stackoverflow.com/questions/3305413/python-strptime-and-timezones.

In particular, these fail when include UTC: '%a %b %m %X %y %z' and '%c %z'.

I am resorting to a utility package.

In [21]:
import datetime
from dateutil import parser
parser.parse("Tue Aug 2 14:58:31 2016 +0200")  #+0200 hours => 7200 seconds

datetime.datetime(2016, 8, 2, 14, 58, 31, tzinfo=tzoffset(None, 7200))

In [22]:
all_commits = []
for entry in tups:
    name = entry[0]                               #'Gilles Fourestey <gilles.fourestey@epfl.ch>'
    commit_list = entry[1]['commits']
    for a_commit in commit_list:
        date = parser.parse(a_commit['date'])         #"Tue Aug 2 14:58:31 2016 +0200"
        files = []
        diffs = a_commit['diffs']
        for diff_dict in diffs:
            filename = diff_dict['filename']          #'var/spack/repos/builtin/packages/plumed/package.py'
            files.append(filename)
        message = a_commit['message']             #'Fixed broken link in README (#1733)\n'
        new_dict = {'date': date, 'files': files, 'message': message, 'person': name}
        all_commits.append(new_dict)
        
    

In [23]:
len(all_commits) #2756 commits spread over roughly 3 years

23059

In [24]:
all_commits[0]

{'date': datetime.datetime(2016, 9, 6, 14, 52, 44, tzinfo=tzoffset(None, -25200)),
 'files': [',7 +20,7 @@ written in pure Python, and specs allow package authors to write a'],
 'message': '    Fixed broken link in README (#1733)\n',
 'person': 'Ian Lee <IanLee1521@gmail.com>'}

Looks ok to me. Now converting to pandas dataframe is easy peasy.

In [25]:
import pandas as pd

In [26]:
df_commits = pd.DataFrame(all_commits)  #gotta love pandas

In [27]:
df_commits.head()

,date,files,message,person
0,2016-09-06 14:52:44-07:00,"[,7 +20,7 @@ written in pure Python, and specs...",Fixed broken link in README (#1733)\n,Ian Lee <IanLee1521@gmail.com>
1,2016-09-06 14:52:44-07:00,"[,7 +20,7 @@ written in pure Python, and specs...",Fixed broken link in README (#1733)\n,Ian Lee <IanLee1521@gmail.com>
2,2018-06-28 13:32:43-07:00,"[,7 +31,8 @@ class Nmap(AutotoolsPackage):]",Added nmap version 7.70 (#8606)\n,Ian Lee <IanLee1521@gmail.com>
3,2016-09-06 14:52:44-07:00,"[,7 +20,7 @@ written in pure Python, and specs...",Fixed broken link in README (#1733)\n,Ian Lee <IanLee1521@gmail.com>
4,2016-08-02 14:58:31+02:00,[var/spack/repos/builtin/packages/plumed/packa...,added plumed\n \n * plumed : first v...,Gilles Fourestey <gilles.fourestey@epfl.ch>


In [ ]:
#df_sorted = df_commits.sort('date')  #this fails in 2.7

In [28]:
df_sorted = df_commits.sort_values(by='date')
df_sorted = df_sorted.reset_index(drop=True)

In [29]:
df_sorted.head()

,date,files,message,person
0,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
1,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
2,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
3,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>
4,2013-02-13 17:50:44-08:00,"[.gitignore, bin/spack]",Initial version of spack with one package:...,Todd Gamblin <tgamblin@llnl.gov>


In [30]:
df_sorted.tail()

,date,files,message,person
23054,2018-12-31 23:40:32-08:00,"[2,6 +122,6 @@ See [LICENSE-MIT](https://githu...",README: LLNL release number and SPDX id do...,Todd Gamblin <tgamblin@llnl.gov>
23055,2019-01-01 09:11:49+01:00,"[2,6 +562,9 @@ class Singleton(object):]",Improve validation of modules.yaml (#9878)...,Massimiliano Culpo <massimiliano.culpo@gmail.com>
23056,2019-01-01 06:05:53-08:00,"[,6 +14,7 @@ class RReadr(RPackage):]","Add list_url for r-readr, finding old vers...",George Hartzell <hartzell@alerce.com>
23057,2019-01-02 00:39:07+08:00,"[,7 +58,9 @@ class Fenics(CMakePackage):]",FEniCS: workaround concretization issue wi...,Ben Zwick <33216696+benzwick@users.noreply.git...
23058,2019-01-02 01:15:15+01:00,"[,7 +78,7 @@ module_file_configuration = {]",Removed an extra comma after a dict litera...,Massimiliano Culpo <massimiliano.culpo@gmail.com>


In [31]:
len(df_sorted)

23059

Write it out to file

In [32]:
save_dir = repository_dir + '/machine_learning/predicting-daily-commits/'

In [33]:
df_sorted.to_csv(save_dir + 'sorted_commits.csv')  #ideas-uo-master/code/tmp/spack